In [ ]:
!pip install tensorflow-probability

In [ ]:
!pip install --upgrade lida==0.0.10 fastapi pydantic pydantic-core

In [ ]:
! pip install pandas
! pip install openai==0.28.1
! pip install gradio

In [ ]:
import pandas as pd
import openai
import numpy as np
import gradio as gr
import pickle
from sklearn.metrics.pairwise import cosine_similarity
from openai.embeddings_utils import get_embedding

In [3]:
openai.api_key = 'Your-openai-key'

In [4]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/embedding-movie-recommender/large_movie_data.csv') # we define df to read our csv file.

In [6]:
# in the df we define a column name "Embedding" to store the embedded value of "Movie" column. the lambda function goes through each line of Movie column and the
# ada module creates the embeddings.
# we use the second line 'with open('file_path') to store the pickle file.
# the function f:pickel.dump(df,f) is the function that creates embedding vectors. it takes about 12 miniutes to complete.
df['Embedding'] = df['Movie'].apply(lambda x: get_embedding(x, engine='text-embedding-ada-002'))
with open('/content/drive/MyDrive/Colab Notebooks/embedding-movie-recommender/large_movie_data_embedded.pkl', 'wb') as f:
  pickle.dump(df, f)

In [4]:
df = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/embedding-movie-recommender/large_movie_data_embedded.pkl') # we use df.read_pickle('file_path') to read the pickle file. To see it, we need to convert it to CSV.
df.to_csv('/content/drive/MyDrive/Colab Notebooks/embedding-movie-recommender/large_movie_data_embedded.csv')

In [ ]:
# now we need to load our pickle file.
with open('/content/drive/MyDrive/Colab Notebooks/embedding-movie-recommender/large_movie_data_embedded.pkl', 'rb') as f:
   df = pickle.load(f)

In [27]:
# now that we loaded our embedded data, we create a search function to test our model.
# the function 'search_movies' compares df with movie_title (user input from gradio). then n=2 picks second most similar title(first one is the movie title itself we input.)
def search_movies(df, movie_title, n=2):
  # embedding module creates embedding of movie_title (user input) using the same engine.
  embedding = get_embedding(movie_title, engine='text-embedding-ada-002')
  # df creates 'similarities' column where it stores similarity value of 'embedding' or (user input movie title) and 'Embedding' or the Movies embedded values.
  df['similarities']= df.Embedding.apply(lambda x: cosine_similarity([x], [embedding]))
  # store the results in a df called res. res contains the top n values in descending order.
  res = df.sort_values('similarities', ascending=False).head(n)
  # return the searched Movie title and its corresponding similarity value.
  return res.iloc[1]['Movie'], res.iloc[1]['similaritis']

modifications: in below gradio code, I removed the "Interpretation='default', to solve the error. in the new version of gradio, this line is causing a bug.

In [ ]:
# Define Gradio interface for the recommendation system
def gradio_wrapper(movie_title):
    top_movie, similarity_score = search_movies(df, movie_title)
    return top_movie, similarity_score

iface = gr.Interface(
    fn=gradio_wrapper,
    inputs="text",
    outputs=["text", "number"],
    #interpretation="default",
)
iface.launch(share=True)